In [1]:
%config ZMQInteractiveShell.ast_node_interactivity = "all"
%pprint

Pretty printing has been turned OFF


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import sys
import torch
import numpy as np
import torch.nn as nn
sys.path.append("../d2l_func/")
from data_prepare import load_data_jay_song, data_iter_random, data_iter_consecutive, to_onehot
from model_train import train_rnn, train_rnn_pytorch
from set_seed import set_seed
set_seed(2020)

## LSTM

LSTM有输入门，遗忘门，输出门
- 输入门：$i_t = \delta(x_tw_{xi} + h_{t-1}w_{hi} + b_i)$
- 遗忘门：$f_t = \delta(x_tw_{xf} + h_{t-1}w_{hf} + b_f)$
- 候选元胞状态：$\widetilde c = tanh(x_tw_{xc} + h_{t-1}w_{hc} + b_c)$
- 元胞状态：$c_t = i_t*\widetilde c + o_t*c_{t-1}$
- 输出门：$o_t = \delta(x_tw_{xo} + h_{t-1}w_{ho} + b_o)$
- 隐藏层状态：$h_t = o_t*tanh(c_t)$

### 自定义实现

#### 定义网络参数

In [4]:
def get_params(input_num, hidden_num, output_num, device):
    def _ones(shape):
        weight = nn.Parameter(torch.normal(0, 0.01, size=shape, device=device), requires_grad=True)
        return weight
    
    def _zeros(shape):
        bias = nn.Parameter(torch.zeros(shape, device=device), requires_grad=True)
        return bias
    
    def _three():
        return (
            _ones((input_num, hidden_num)),
            _ones((hidden_num, hidden_num)),
            _zeros(hidden_num)
        )
    
    # 输入门/遗忘门/输出门
    w_xi, w_hi, b_i = _three()
    w_xf, w_hf, b_f = _three()
    w_xo, w_ho, b_o = _three()
    # 元胞状态
    w_xc, w_hc, b_c = _three()
    # 输出层
    w_hy = _ones((hidden_num, output_num))
    b_y = _zeros(output_num)
    
    return nn.ParameterList([w_xi, w_hi, b_i, w_xf, w_hf, b_f, w_xo, w_ho, b_o, w_xc, w_hc, b_c, w_hy, b_y])

#### 定义网络结构

In [5]:
from functools import reduce

def init_hidden_state(batch_size, hidden_num, device):
    return (torch.zeros(batch_size, hidden_num, device=device), 
            torch.zeros(batch_size, hidden_num, device=device))


def lstm(inputs, h_state, params):
    w_xi, w_hi, b_i, w_xf, w_hf, b_f, w_xo, w_ho, b_o, w_xc, w_hc, b_c, w_hy, b_y = params
    outputs = []
    h, c = h_state
    
    # inputs.shape is (num_step, batch_size, vocab_size)
    for x in inputs:
        it = torch.sigmoid(torch.mm(x, w_xi) + torch.mm(h, w_hi) + b_i)
        ft = torch.sigmoid(torch.mm(x, w_xf) + torch.mm(h, w_hf) + b_f)
        ot = torch.sigmoid(torch.mm(x, w_xo) + torch.mm(h, w_ho) + b_o)
        c_candidate = torch.tanh(torch.mm(x, w_xc) + torch.mm(h, w_hc) + b_c)
        c = it*c_candidate + ft*c
        h = ot*torch.tanh(c)
        y = torch.mm(h, w_hy) + b_y
        outputs.append(y.unsqueeze(0))
        
    return reduce(lambda x, y: torch.cat((x, y)), outputs), (h, c)

In [6]:
# 验证
hidden_num, vocab_size, device = 10, 15, "cuda"
x = torch.arange(10).view(2, 5)
inputs = to_onehot(x, vocab_size, device)
h_state = init_hidden_state(inputs.shape[1], hidden_num, device)
params = get_params(vocab_size, hidden_num, vocab_size, device)
outputs, h_state = lstm(inputs, h_state, params)
# 输出/隐藏状态/元胞状态
outputs.shape, h_state[0].shape, h_state[1].shape

(torch.Size([5, 2, 15]), torch.Size([2, 10]), torch.Size([2, 10]))

#### 预测

In [7]:
def predict_rnn(prefix, pred_num, model, init_hidden_state, hidden_num, 
                params, char_to_idx, vocab_set, vocab_size, device):
    outputs = [char_to_idx[prefix[0]]]
    h_state = init_hidden_state(1, hidden_num, device)
    
    for i in range(len(prefix) + pred_num - 1):
        # inputs.shape is (1, 1, vocab_size)
        inputs = to_onehot(torch.tensor(outputs[-1]).view(-1, 1), vocab_size, device)
        # y.shape is (1, 1, vocab_size), h_state.shape is (1, hidden_num)
        y, h_state = model(inputs, h_state, params)
        
        if i + 1 < len(prefix):
            outputs.append(char_to_idx[prefix[i+1]])
        else:
            outputs.append(y.argmax(dim=2).item())
            
    return "".join(vocab_set[i] for i in outputs)

In [8]:
# 验证
# load data
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()
# params
hidden_num, device = 256, "cuda"
params = get_params(vocab_size, hidden_num, vocab_size, device)
predict_rnn("分开", 10, lstm, init_hidden_state, hidden_num, params, char_to_idx, vocab_set, vocab_size, device)

'分开队局挽仁还掰汰钥唤幻'

#### 训练

In [9]:
# load data
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()

super_params = {
        "epoch_num": 250,
        "rnn": lstm,
        "loss": nn.CrossEntropyLoss(),
        "init_hidden_state": init_hidden_state,
        "hidden_num": 256,
        "get_params": get_params,
        "batch_size": 64,
        "num_step": 32,
        "corpus_index": corpus_index,
        "data_iter": data_iter_random,
        "lr": 100,
        "char_to_idx": char_to_idx,
        "vocab_set": vocab_set,
        "vocab_size": vocab_size,
        "predict_rnn": predict_rnn,
        "pred_num": 50,
        "prefixs": ["分开", "不分开"],
        #     "random_sample": False
    }

super_params["batch_num"] = len(list(data_iter_random(corpus_index, super_params["batch_size"],
                                                      super_params["num_step"], "cpu")))

train_rnn(**super_params)

Epoch [1/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 1310.3052, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [2/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 1924.4729, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [3/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 1574.2936, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开的的  我  我  我  我  我  我  我  我  我  我  我  我  我  我  我  我
prefix-不分开:  不分开的的  我  我  我  我  我  我  我  我  我  我  我  我  我  我  我  我


Epoch [4/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 2315.3799, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                         

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 371.1947, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的灵变 我们中一种调调　 敲风　敲飘　我的自己 你的微笑 我们在一点调调飘移 那些年的人会有 功
prefix-不分开:  不分开 我知道 你的嘴变在你旁边的美丽 你的完美 我们中一种调调　 敲风　敲飘　我的自己 你的微笑 我们在


Epoch [34/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 345.5335, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我想我 我们 我想                                        
prefix-不分开:  不分开 我知道你 一场 我们在一起                                    


Epoch [35/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 322.3889, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
prefix-不分开:  不分开 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我


Epoch [36/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 300.9271, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的爱 别人在台 我对着你的一口 想说的出爱 我会微笑着 你好                  
prefix-不分开:  不分开 我会一直好好 你的手不放开 爱可不可以简简单单没有伤害 我会发着呆 你的爱情 

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 71.4072, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我说你爱我 开不了别让你知道 我没有紧紧也住你会 爱你比你的大离 就算是谁的口 有些种味道到底是你
prefix-不分开:  不分开 我知道共同的默契很重要 那骄傲才不会寂寞的很无聊　 那骄傲才不会寂寞的很无聊　 那骄傲才不会寂寞的


Epoch [66/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 69.0984, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我对着流星 一统江山 破胜之中 我却微笑绝不恋战 我对着这些奏 后知后觉 又过了一个秋 后知后觉 
prefix-不分开:  不分开 我知道共同一点称 那骄地心碎翼的笑 就怎么面 我的感觉 你不会感来 有专事没太快 我会发着呆   


Epoch [67/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 66.7813, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我也能够能不出来 我不会再想你 我知道你眼泪没有错 只能回忆 泥泞迎甲我挥泪杀 满城菊花谁的天空 
prefix-不分开:  不分开 我知道共同的默契很重要 我知道共同的默契很重要 那骄傲才不会寂寞的很无聊　 也知道不要的分手 说不


Epoch [68/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 64.5327, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我也想要我 一场 强壮 我想起 你的终一笑 我的世界被你拆封 说散了别多 我们在直离离欣 我对着你
prefix-不分开:  不分开 我知道共同的默契很重要 那些年才不会寂寞的很无聊　 那骄傲才不会寂寞的很无聊　 那骄傲

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 30.8746, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我对我有多难熬  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼  没有你烦我有多烦恼多难熬  
prefix-不分开:  不分开 我就会感情 加目光 如果说分手是一种解剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的回忆 没有


Epoch [98/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 30.2660, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我说你爱 你就像豆腐 豆腐  我在武功学校里学的那叫功夫 功夫 功夫  功夫 功夫  赶紧穿上旗袍
prefix-不分开:  不分开 我知道共同一点汗　你会微笑很我心里 你说好不好 你的笑 你说好好好难心 我不会等待走   你是我的


Epoch [99/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 29.6718, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我对我的爱写你拆走 你默背为我掉过几次泪多憔悴 而我心碎你受罪你的美 我不配 这街上太拥挤 太多人
prefix-不分开:  不分开 我对着流星 却只想你和汉堡 我想要你的微笑每天都能看到  我知道这里很美但家乡的你更美走过了很多地


Epoch [100/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 29.0937, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我对我的爱写你拆走 你的笑容我不能够说我们一定就好  没有你在我有多难熬多烦恼  没有你烦 我有多
prefix-不分开:  不分开 我知道共同一遍 你要的爱我 舍不得吃会微笑的糖果 我轻轻地尝一口 份量虽然不多 我将

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 18.1608, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 让我连恨都难以下笔 将真心抽离写成日记 像是一场默剧 你的完美主义 太彻底 
prefix-不分开:  不分开 我就算自己心掉 我们拥有同一边调 这样的名到 谁都了我疲我 我还在等待模仿 如果我遇见你是一场悲剧


Epoch [130/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.9256, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后在这里 不限日期 
prefix-不分开:  不分开 我知道你的痛没有 载起 我想大再怎么面也  你叫你们眼球 开不到口　 要多到过去 一直一直向直 旧


Epoch [131/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.6856, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美 我微笑的感动 我只取你 转指 哭 生命就该完整的冰 在窗外头之中 我们的思念 还有是你的
prefix-不分开:  不分开 我在等你灵魂 温柔中慈祥 在我地盘这   你就得听我的 把音乐收割   用听觉找快乐 开始在雕刻 


Epoch [132/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.4522, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美 我微笑的笑像 你站的脸位 你我我我该是陪了天 化身为龙 那大地心脏汹涌 不安跳动 全世界
prefix-不分开:  不分开 我会将会哭人 的B魂 翻滚 停止忿恨 永无止尽的战争 让我们 半兽人 的灵魂 单

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 12.5088, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 因為了依赖 没关系 慢
prefix-不分开:  不分开 我知道共同的默契很重要 我知道共同的默契很重要 我知道共同的默契很重要 我知道共同的默契很重要 我


Epoch [162/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 12.3841, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 让我连恨都难信下笔 将真心抽离写成日记 像是一场默剧 你的完美主义 太彻底 
prefix-不分开:  不分开 我知道不能再留住你 你会当离离开口气 如果说分手是苦痛的起点 那在终点之前   我愿意再爱一遍 想


Epoch [163/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 12.2637, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后跟了你 谁在用琵琶
prefix-不分开:  不分开 我知道共同 走 你眼睛一直离会一起 梦想就对着 我们的那怎么才微笑 你对着的香方 微笑着远的相爱 


Epoch [164/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 12.1423, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无法为无重 我变不得离开 看不见你的笑我怎么睡得着　
prefix-不分开:  不分开 我知道共同的默契很重要 我知道共同的默契很重要 那骄傲才不会寂寞的很无聊　 书框

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 9.4491, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 我对着流星 祈祷\时专心 為爱飞行 脱离地心引力的热情 我在宇宙无重力的环境 為你降临 赤道的边境
prefix-不分开:  不分开 我知道共同的默契是否褪色想问你的脸色 我是你的废物 一把锥 一起一颗往外爬 到底看着泪海里 一起上


Epoch [194/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 9.3766, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 没有了证明 没有了空虚 基于两种立场我会罩着你 趁时间没发觉 让我带着你离开 这不是顽固 这不是逃
prefix-不分开:  不分开 我永远 都想要一个人 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟


Epoch [195/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 9.3042, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的香味一直徘徊 我舍不得离开 有谁能比我知道 你的微笑像拥抱 多想藏着你的好 只有我看得到 有些
prefix-不分开:  不分开 老师在战 我是前的字好 谁想起你却累不去我的细气　 也可以风气如重新给我们 我不能够后退 因为我两


Epoch [196/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 9.2373, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的灵魂属于我 来找我 找不到我 你那迷路的眼眸 跟着我 被我诱惑 众神都已经着了魔 说爱我 爱不
prefix-不分开:  不分开 老师在台 我听了你追 我说著头  M後颐I 你的美 已经给了谁 追了又追 我要不回 我了

31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 7.5761, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 然后忘了 离开薄你的美
prefix-不分开:  不分开 没有你以后 我知道你们都没有错 只是忘了怎么退后 信誓旦旦给了承诺 却被时间扑了空 我知道我们都没


Epoch [226/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 7.5297, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再提起 决定中断熟悉 转身离开  分手的不能
prefix-不分开:  不分开 老师再来 你离我还多难回  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼  没有你烦我有多烦恼


Epoch [227/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 7.4840, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你说我像一个小孩 总爱让你猜 我说你才像个小孩 总要我说才明白 有些事太快 失去了等待 让爱没了期
prefix-不分开:  不分开 我在等你 炊烟袅袅升起 隔江千万里 在瓶底书刻隶 仿前朝的飘逸 就当我为遇见你伏笔 天青色等烟雨 


Epoch [228/250]
31/31 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 7.4379, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开 你说我不该不该 不该在这时候说了我爱你 要怎么证明我没力气 可是我只有一天的回忆 你说我不该不该 
prefix-不分开:  不分开 泪快将了最后 大世界蔓月 老板在冬天 在华丽的小季被 却彷不回来 和平这张近 又要了一个

### 简洁实现

#### 网络结构

In [4]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_num = self.rnn.hidden_size * (2 if self.rnn.bidirectional else 1)
        self.vocab_size = vocab_size
        self.fc = nn.Linear(self.hidden_num, vocab_size)
        self.h_state = None
        
    def forward(self, x, h_state):
        y, self.h_state = self.rnn(x, h_state)
        return self.fc(y), self.h_state

#### 预测

In [5]:
# 预测
def predict_rnn_pytorch(prefix, pred_num, model, char_to_idx, vocab_set, vocab_size, device):
    outputs = [char_to_idx[prefix[0]]]
    h_state = None
    
    for i in range(len(prefix) + pred_num - 1):
        inputs = to_onehot(torch.tensor(outputs[-1]).view(-1, 1), vocab_size, device)
        if h_state is not None:
            if isinstance(h_state, tuple): # lstm , (h,c)
                h_state = (h_state[0].to(device), h_state[1].to(device))
            else:
                h_state = h_state.to(device)
                
        y, h_state = model(inputs, h_state)
        if i + 1 < len(prefix):
            outputs.append(char_to_idx[prefix[i+1]])
        else:
            outputs.append(y.argmax(dim=2).item())
            
    return "".join(vocab_set[i] for i in outputs)

In [6]:
# 验证
# load data
hidden_num = 256
corpus_index, char_to_idx, vocab_set, vocab_size = load_data_jay_song()
rnn_layer = nn.LSTM(vocab_size, hidden_num)
model = RNNModel(rnn_layer, vocab_size)
model = model.cuda()
predict_rnn_pytorch("分开", 10, model, char_to_idx, vocab_set, vocab_size, "cuda")

'分开传传传感彿传传感彿传'

#### 训练

In [8]:
hidden_num = 256
rnn_layer = nn.LSTM(vocab_size, hidden_num)
model = RNNModel(rnn_layer, vocab_size)
model = model.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

params = {
    "epoch_num": 250,
    "model": model,
    "loss": loss,
    "optimizer": optimizer,
    "batch_size": 64,
    "num_step": 32,
    "corpus_index": corpus_index,
    "data_iter": data_iter_consecutive,
    "char_to_idx": char_to_idx,
    "vocab_set": vocab_set,
    "vocab_size": vocab_size,
    "predict_rnn_pytorch": predict_rnn_pytorch,
    "pred_num": 50,
    "prefixs": ["分开", "不分开"],
    "random_sample": False
}

params["batch_num"] = len(list(data_iter_consecutive(corpus_index, params["batch_size"],
                                                     params["num_step"], "cpu")))

train_rnn_pytorch(**params)

Epoch [1/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 776.9311, train_score: -, test_loss: -, test_score: --

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [2/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 563.4801, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [3/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 500.5215, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                                                  


Epoch [4/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 469.6437, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开                                                  
prefix-不分开:  不分开                            

Epoch [33/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 190.1486, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 我不能再想 你说你的爱情 我不能不能 我 你的爱情 我不能不能 我 你的爱情 我不能不能 我
prefix-不分开:  不分开 我不能不能 我们 我们一个人 我们的爱情 我不能不能 我不能不能 我们 我们一个人 我们的爱情 我


Epoch [34/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 183.5192, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 我不能再想 我们 这样子的手 我不要我不能 我不能不能 我不要我 我不能不能 我不要我 我不
prefix-不分开:  不分开 我不能再多 我 我不能不能 我们 我们的爱情 我不能不能 我 你的爱情 我不能不能 我 你的爱情 


Epoch [35/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 177.2048, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 我不能再想 你说你说你不能 我不能再想 你 我不能不能 我 我不能不能 我 你说你的爱情 我
prefix-不分开:  不分开 我不能再见 我 我不能不能 我 我不能不能 我 你说你的爱情 我不能不能 我 你说你的爱情 我不能


Epoch [36/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 171.1141, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 我不能再想 你说你的爱情 我不能不能再想 我们都不能够不要 我们都不能够不要 我们都不能够不
prefix-不分开:  不分开 我不能再见 我 我不能再见 我 你不要再想 我 

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 69.0070, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能不能 我们一个人生  不开 不能再见 我不能再见 你的手不放开 不能再想你 我的爱情 就像
prefix-不分开:  不分开始不能 能够能够远方的手 一场悲剧 我我在等待 心 我的世界一幕的画面 天空灰过去过过 过过过去过过


Epoch [66/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 67.1355, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能够能够远方 一一种味道  用古老斑驳的画面 谁在月光下来 在我们乘着阳光　 等待英雄也累了
prefix-不分开:  不分开始不能 能够能够远方的手 一场悲剧 我我的爱情绪  不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 


Epoch [67/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 65.3439, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能不能 我们一个人们都不来 我不能再想 你的手不放开 不能再想一个我的感觉 你说我的可爱女人
prefix-不分开:  不分开始不能 能够能不能再一个人生气 这样子不会太多 我不能再想你一首歌的感动 不能再想你 我们一直走过 


Epoch [68/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 63.5846, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能不能 我们一直 是我的手 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用
prefix-不分开:  不分开始不能 能够能够远远的爱上  一直到过去过去 一起上白 在小村外 溪边河口 默默支着你们

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 32.0045, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能够能够远 是否在风吹 天空气 想要这么简单你 不要我 想念你的我  我们的感觉 我不开 让
prefix-不分开:  不分开始不回 失去 你会有人开始来不及 不该 不该 不该 不该 不该 我们 这个人 听妈的 音乐开始的人生


Epoch [98/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 31.3600, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能够能够远 是否在江水里 你的声音 我用第一人称在飘移青春  输跟赢的分寸计算很很准  我踏
prefix-不分开:  不分开始不能 能够能够远 是否在江湖上 一直往前我的泪滴  车林里 不用第六百公主人们都睡 着一种味道 你


Epoch [99/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 30.7257, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读着对白  用心里写的日记 口袋里了糖 黄金葛爬满了雕花的漫画 一阵地方 一起长大的满
prefix-不分开:  不分开始不能 能够能永远 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦


Epoch [100/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 30.1118, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读着对白  用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 不用
prefix-不分开:  不分开始不能 能够能永远 是人呢 我的手 我开始地狱 叛军如野火般攻击 五官差点离开身体  

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.9800, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读你的爱写 开始不会掩埋在美 那么会扯去扯铃　 扯多你就会上瘾 扯你最善变的表情　 嘿
prefix-不分开:  不分开始不能 能够能不能 我一口 我们一起来 我的爱情悬 你说你会我会陪你走过 你发生 放手 你们躲过的 


Epoch [130/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.7007, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读依依旧每天都没有预料 竟然先对我示好 她的睫毛弯的嘴角 用眼神对我拍照 我戒不掉她的
prefix-不分开:  不分开始不能 能够能不能 这样子 太多问我一生活活 我一定再爱的讯号  总有些事听我的感觉 你说好不开 对


Epoch [131/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.4319, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能别想太多 没有没人帮你擦眼泪 别离开身边拥有你我的世界才能保护她 美丽的白发 幸福中发芽 天使
prefix-不分开:  不分开始不能 能够能不能 因為我 越 梦朦的飞月 心 如果华佗再人的世界 被它不掉 泪水在蓝天份门 记忆你


Epoch [132/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 17.1669, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读你读你 一起旅行 那童年的希望是 一台 时光机 我一定一直都猜在  一直到老了一口 
prefix-不分开:  不分开始不能 能够能一直走到最后 不该在一生 再想一个  没有一直 我有一个人 心 最后

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 11.6385, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能再重来 你发香味 爱上的爱情 回忆里的画 我说了 这个世 已经远离开 唱的话叫 你用卡 都不要
prefix-不分开:  不分开了了什么 静 让我们乘着阳　 静静看守这份量　雨样也得很重　 快疯掉　记录自己记得你记得你 我知道　


Epoch [162/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 11.5033, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读你的可爱 好像我有了吧 你说我不该 睁开心 别太多 多难熬  没有在意义 专心的完全
prefix-不分开:  不分开了叫爱 把还能给会更人 一个人称　在飘移青春 就该完成熟坦白  不换气 热情为我 你的生活  你说啊


Epoch [163/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 11.3688, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读你对手 我的指尖 阻止我说再见 是谁在风中 面红的可爱女人 温柔的让我们遇同的梦境在
prefix-不分开:  不分开了叫爱 把所有没有公平 才能够明白 走是开草的爱情给你 我会发著呆 然後忘记你  接著紧紧闭上眼 想


Epoch [164/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 11.2369, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 能永远读你对手 我背决定难道理 那童年的梦的画  一出就是一场走 都有眼泪 可能再提一只停左
prefix-不分开:  不分开了叫爱 把所有没有办法 我表情好好 好难过雨 最后才明白 有一个 梦里找了一行 花

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.2993, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能别离开 你说的爱情 当作我最后才明白 看不是你爸的那祝我 舍得飘得得很远 离开 没人喜欢被我喜
prefix-不分开:  不分开了叫默 把手牵手出手写 小小心里 天真的让我颠倒下心 那一天真在抽方 那发香 这幸福 别发如雪 是我


Epoch [194/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.2226, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能别离开 你说的爱上 只是我错过的请你当我会发现在 干什么 放不来 不能再见 这里话的思念像还是
prefix-不分开:  不分开了了什么 我 想这首止的地方 你会离开 跑不了再说 我爱我的爱我在 这里你的人像一场 人生去年前球 


Epoch [195/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.1471, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能别离开 你说的爱情有才能比别人快也会怕走到这么能就能别人在身边的身体　 看看的颜色我怀抱你在怀
prefix-不分开:  不分开了叫默 不能再想 这样的好生 还是不自导 时间快乐 我 他们精武出手无人能躲 我戒不掉她我的狗 啦～


Epoch [196/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 8.0724, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能停留 因为在爱你上的下道在你身边 断了 想念你说我 陪我跑挖草 学习摆弄它 学生宿舍空荡荡的角
prefix-不分开:  不分开了了什么 也不会怎么会比我更暖 求属于上的白  转弯着像顶不能别想太快疯掉 爱你要我加美 

30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.3632, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 怕你要你已表现在那里面对 天空回头绕 大人们始终于拼上上的灯　 在飘移 差点滴答滴答案是刚刚
prefix-不分开:  不分开了了什么 也不会弹做自人的秘密 我是宇宙看看 你来会慢慢习 这么这样忍着痛泪也不会用几重要这么简单你


Epoch [226/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.3167, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 怕你心碎你知道你的微笑像哭  加看见 才到眼睛 还是开了天就是朋友掉 泪在飘移 有点 当年以
prefix-不分开:  不分开了了什么 也不会换来太快在痛 是不是在我手上的歌谣 赶紧紧 怎么记得 记得你叫我 你说你才能完她受伤


Epoch [227/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.2702, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能 怕你撑不是因为我掉过 分手说不出 再来 这样好 对着对话说再见 难过的没人帮着甜蜜的离开 有
prefix-不分开:  不分开了了什么 也不会绕过去气　 会让人能失忆 能不能 说永远做不到  你会一直走 你说不再开Picke 


Epoch [228/250]
30/30 [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] - train_loss: 6.2246, train_score: -, test_loss: -, test_score: -

prefix-分开:  分开始不能别离开 全身带汉  你说我不懂 整夜 这里的歌词 几个字报　 穿了时代 如果邪恶空是我到解球 
prefix-不分开:  不分开了了什么 也不会绕过去意义 将往的让我有你比较好  而过了这种身份 几个飘移 有点心跳　别